In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


In [10]:
# first country in df
df.sort_index()
df.index[0]
df.iloc[0]

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

In [22]:
# which country has most gold in summer games
def ans():
    df1 = df.copy()
    df1['Country'] = df.index
    df1.reset_index()
    df1 = df1.set_index(['Gold'])
    df1 = df1.sort_index()
    length = len(df1.index)
    return df1.iloc[length-1]['Country']

ans()

'United States'

In [45]:
# which country has biggest difference between their summer and winter gold medal counts?
def sub(a,b):
    return abs(a-b)

def ans():
    df2 = df.copy()
    difference = list(map(sub,df2['Gold'],df2['Gold.1']))
    df2['Diff'] = difference
    df2 = df2[['Gold','Gold.1','Diff']]
    df2['Country'] = df2.index
    df2.reset_index()
    df2 = df2.set_index(['Diff'])
    df2 = df2.sort_index(ascending=False)
    return df2.iloc[0]['Country']

ans()
    

'United States'

## 3 

Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count?

(Summer Gold−Winter Gold)/Total Gold

Only include countries that have won at least 1 gold in both summer and winter.


In [71]:
def add(a,b):
    return a+b

def rel(a,b,c):
    return (a-b)/c

def ans():
    df3 = df.loc[:,['Gold','Gold.1','Gold.2']]
    df3 = df3.where((df['Gold']>=1) & (df3['Gold.1']>=1))
    df3 = df3.dropna()
    df3['Rscore'] = list(map(rel,df3['Gold'],df3['Gold.1'],df3['Gold.2']))
    df3['Country'] = df3.index
    df3.reset_index()
    df3 = df3.set_index(['Rscore'])
    df3 = df3.sort_index(ascending=False)
    return df3.iloc[0]['Country']

ans()

'Bulgaria'

# 4

Write a function that creates a Series called "Points" which is a weighted value where each gold medal (Gold.2) counts for 3 points, silver medals (Silver.2) for 2 points, and bronze medals (Bronze.2) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

This function should return a Series named Points of length 146

In [73]:
def add(a, b, c):
    return 3*a+2*b+c

def ans():
    df4 = df.loc[:,['Gold.2','Silver.2','Bronze.2']]
    total = list(map(add,df4['Gold.2'],df4['Silver.2'],df4['Bronze.2']))
    series = pd.Series(total,index=df4.index)
    return series

ans()

Afghanistan                           2
Algeria                              27
Argentina                           130
Armenia                              16
Australasia                          22
                                   ... 
Yugoslavia                          171
Independent Olympic Participants      4
Zambia                                3
Zimbabwe                             18
Mixed team                           38
Length: 146, dtype: int64


## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.



In [162]:
census_df = pd.read_csv('census.csv')



,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
314,40.0,1.0,1.0,9.0,0.0,Connecticut,Connecticut,3574097.0,3574118.0,3579717.0,...,-3.724400,-5.311208,-4.730271,-7.567093,-7.687268,0.775510,-1.066084,-0.278693,-2.376831,-2.463243
315,50.0,1.0,1.0,9.0,1.0,Connecticut,Fairfield County,916829.0,916850.0,919744.0,...,-0.228261,-3.030708,-2.070291,-6.633789,-7.385939,6.119788,2.687406,4.096896,0.441829,-0.338989
316,50.0,1.0,1.0,9.0,3.0,Connecticut,Hartford County,894014.0,894029.0,895303.0,...,-5.037297,-6.324143,-5.237664,-8.636074,-8.135161,-0.025666,-1.662136,-0.314193,-2.840433,-2.327200
317,50.0,1.0,1.0,9.0,5.0,Connecticut,Litchfield County,189927.0,189927.0,189773.0,...,-4.103317,-7.183847,-2.812353,-7.951141,-5.931948,-3.290047,-6.450584,-1.983618,-6.939080,-4.906204
318,50.0,1.0,1.0,9.0,7.0,Connecticut,Middlesex County,165676.0,165676.0,165628.0,...,1.410378,-6.716791,-2.617825,-4.987002,-6.526367,3.598271,-4.522077,-0.308335,-2.272328,-3.807554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3164,50.0,2.0,3.0,55.0,133.0,Wisconsin,Waukesha County,389891.0,389938.0,390076.0,...,-0.765799,2.128860,0.038132,0.760109,-0.719858,0.102448,3.180527,1.189727,2.077633,0.593567
3165,50.0,2.0,3.0,55.0,135.0,Wisconsin,Waupaca County,52410.0,52410.0,52422.0,...,3.111756,-2.241873,6.292687,-0.441031,-0.480617,3.359933,-2.011937,6.561277,-0.134227,-0.173022
3166,50.0,2.0,3.0,55.0,137.0,Wisconsin,Waushara County,24496.0,24496.0,24506.0,...,4.930022,-2.404973,-4.097017,-4.906711,-4.397793,5.174486,-2.160399,-3.810226,-4.535615,-4.024395
3167,50.0,2.0,3.0,55.0,139.0,Wisconsin,Winnebago County,166994.0,166994.0,167059.0,...,0.316712,2.889873,0.833819,-2.406192,-4.557985,0.842573,3.502335,1.531624,-1.545153,-3.685304


### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [95]:
def ans():
    df5 = census_df.loc[:,['STNAME','COUNTY']]
    df5 = df5.groupby('STNAME').count()
    df5 = df5.sort_values(by=['COUNTY'],ascending=False)
    return df5.index[0]

ans()

'Texas'

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [126]:
def calcpop(count):
    count = count.sort_values(by=['CENSUS2010POP'],ascending=False)
    pop = np.array(count['CENSUS2010POP'])
    pop = np.sum(pop[:3])
    return pop

def ans():
    df6 = census_df.loc[:,['STNAME','COUNTY','CENSUS2010POP']]
    df6.reset_index()
    df6 = df6.set_index(['STNAME'])
    population = []
    states =  df6.index.unique()
    for ind in states:
        population.append(calcpop(df6.loc[ind]))
    series = pd.Series(states,index=population)
    series = series.sort_index(ascending=False)
    return list(series.iloc[:3])

ans()

['California', 'Texas', 'New York']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [202]:
def calc(values):
    values = np.array(values)
    return np.abs(np.max(values)-np.min(values))

def ans():
    df7 = census_df.loc[:,['STNAME','CTYNAME','POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']]
    df7.reset_index()
    df7 = df7.set_index(['STNAME','CTYNAME'])
    #print(df7)
    deviation = []
    for county in df7.index:
            deviation.append(calc(df7.loc[county]))
    df7['Deviation'] = deviation
    df7 = df7.sort_values(by=['Deviation'],ascending=False)
    return df7.index[0][0]

ans()

/home/srinidhi/MyWorks/data_science/lib/python3.7/site-packages/ipykernel_launcher.py:12: PerformanceWarning: indexing past lexsort depth may impact performance.
  if sys.path[0] == '':


'Texas'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [196]:
def ans():
    df8 = census_df.where(((census_df['REGION']==1) | (census_df['REGION'] == 2))).dropna()
    df8 = df8.where((df8['POPESTIMATE2015']>df8['POPESTIMATE2014'])).dropna()
    df8 = df8.where((df8['CTYNAME'].str.contains("Washington"))).dropna()
    df8 = df8.loc[:,['STNAME','CTYNAME']]
    df8 = df8.sort_index()
    return df8

ans()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
